In [1]:
import pygad
import numpy as np
import math

In [122]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        if type(other) is int or type(other) is float:
            return Vector(self.x + other, self.y + other)
        elif type(other) is Vector:
            return Vector(self.x + other.x, self.y + other.y)
        else:
            raise TypeError
    
    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)
    
    def __mul__(self, other):
        return self.x * other.x + self.y * other.y
    
    def angle(self):
        return math.atan2(self.y,self.x) * 180 / math.pi
    
    def distance(self):
        return (self.x**2 + self.y**2)**(0.5)

In [416]:
'''
0 -> free space
1 -> obstacle
2 -> start
4 -> end
'''

class Map:
    def __init__(self, start, end, obstacles, shape = (10,10)):
        self.grid = np.zeros(shape, dtype=np.uint8)
        self.start = start
        self.end = end
        
        self.grid[start[::-1]] = 2
        self.grid[end[::-1]] = 4
        
        for i in obstacles:
            self.grid[i[::-1]] = 1
    
    def get_start(self):
        return self.start
    def get_end(self):
        return self.end
    def get_grid(self):
        return self.grid
    
    def set_start(self, val: (int,int)):
        self.grid[self.start[::-1]] = 0
        self.start = val
        self.grid[start[::-1]] = 2
        
    def set_end(self, val: (int,int)):
        self.grid[self.end[::-1]] = 0
        self.end = val
        self.grid[self.end[::-1]] = 4
    def set_grid(self, val: 'Grid'):
        self.grid = val
    
    def obstacle_threshold(self, start: (int,int), obstacle: (int,int)): # (x,y) returns (low, high)
        obj_vec = Vector(abs(obstacle[0] - start[0]) , abs(obstacle[1] - start[1]))
        
        tl = obj_vec + Vector(-0.5,0.5)
        tr = obj_vec + Vector(0.5,0.5)
        bl = obj_vec + Vector(-0.5,-0.5)
        br = obj_vec + Vector(0.5,-0.5)
        
        
        ls_angle = np.array([i.angle() for i in [tl,tr,bl,br]])

        min_ls_angle = min(ls_angle)
        max_ls_angle = max(ls_angle)
        return (min_ls_angle, max_ls_angle)
    
    def sign(self, x):
        if x>=0:
            return 1
        else:
            return -1

    def obstacle_obstructed(self, start: (int,int), end: (int,int), obstacle: (int,int)):
        traj_vector = np.array([end[0] - start[0], end[1] - start[1]])
        obj_vector = np.array([obstacle[0] - start[0], obstacle[1] - start[1]])

        obj_2_traj = traj_vector - obj_vector

        traj_vector = Vector(end[0] - start[0], end[1] - start[1])
        obj_vector = Vector(obstacle[0] - start[0], obstacle[1] - start[1])
        
        dot = traj_vector * obj_vector

        if dot <= 0: #if it is at or in front of obstacle, then is gud
            return False

        sig_x = self.sign(obj_vector.x)
        sig_y = self.sign(obj_vector.y)
        traj_vector.x *= sig_x
        traj_vector.y *= sig_y

        obj_vector.x *= sig_x
        obj_vector.y *= sig_y

        angle_traj = traj_vector.angle()
        threshold = self.obstacle_threshold(start, obstacle)

        return threshold[0] <= angle_traj <= threshold[1]


In [417]:
class GA_Solver:
    def __init__(self, MAP, generations: int, parents_mating: int, \
                 solutions_per_pop: int, num_nodes: int, 
                 thresh: (int,int), mutation_percent: int):
        self.generations = generations
        self.parents_mating = parents_mating
        self.sol_per_pop = solutions_per_pop
        
        self.num_genes = num_nodes * 2
        (self.init_range_low, self.init_range_high) = thresh
        self.mutation_percent_genes = mutation_percent
        
        self.ga_instance = None
        self.solution = None
        self.solution_fitness = None
        
        self.map = MAP
        
    def fitness_func(self, solution, solution_idx):
        penalty = 0
        solution = list(map(int, solution))
        
        grid = self.map.get_grid()
        
        start = self.map.get_start()
        pts = [tuple(solution[i:i+2]) for i in range(0,self.num_genes,2)]
        end = self.map.get_end()
        
        
        cross_penalty = [9999999999999999 for i in pts if grid[i[::-1]] > 0]
        print(f'incorrect node selection: {len(cross_penalty)}')
        penalty += sum(cross_penalty)
        
        obstacles_idx = np.where(grid==1)
        obstacles_idx = list(zip(obstacles_idx[1], obstacles_idx[0]))
        
        
        dis = 0
        for i in range(len(pts)+1):
            if i == 0:
                vec = Vector(pts[i][0] - start[0], pts[i][1] - start[1])
                #ls_penalty = [99999999 for j in obstacles_idx if self.map.obstacle_obstructed(start,pts[i], j)]
                ls_penalty = []
                for j in obstacles_idx:
                    if self.map.obstacle_obstructed(start,pts[i],j):
                        ls_penalty.append(99999999)
                        print(f'  Obstructed at point {j}.')
                
                print(f'penalty of {start} to {pts[i]}: {len(ls_penalty)}')
                penalty += sum(ls_penalty)
            elif i == len(pts):
                vec = Vector(end[0] - pts[i-1][0], end[1] - pts[i-1][1])
                ls_penalty = [99999999 for j in obstacles_idx if self.map.obstacle_obstructed(pts[i-1],end, j)]
                print(f'penalty of {pts[i-1]} to {end}: {len(ls_penalty)}')
                penalty += sum(ls_penalty)
            else:
                vec = Vector(pts[i][0] - pts[i-1][0], pts[i][1] - pts[i-1][1])
                ls_penalty = [99999999 for j in obstacles_idx if self.map.obstacle_obstructed(pts[i-1],pts[i], j)]
                print(f'penalty of {pts[i-1]} to {pts[i]}: {len(ls_penalty)}')
                penalty += sum(ls_penalty)
            vec_dis = vec.distance()
            if vec_dis == 0:
                penalty += 99999999
            dis += vec_dis
        print()
        return 1.0 / (dis+1) - penalty
        
        
    def train_ga(self):
        def fitness_func(solution, solution_idx):
            return self.fitness_func(solution, solution_idx)
        
        
        gene_range = [{'low': self.init_range_low, 'high': self.init_range_high}] * self.num_genes
        
        self.ga_instance = pygad.GA(num_generations = self.generations,
                                   num_parents_mating = self.parents_mating,
                                   fitness_func=fitness_func,
                                    sol_per_pop=self.sol_per_pop,
                                    num_genes=self.num_genes,
                                   init_range_low=self.init_range_low,
                                    gene_space = gene_range,
                                   init_range_high=self.init_range_high,
                                   mutation_percent_genes=self.mutation_percent_genes)
        self.ga_instance.run()
        solution, solution_fitness, solution_idx = self.ga_instance.best_solution()
        
        self.solution = solution
        self.solution_fitness = solution_fitness
    def convert_solution(self):
        solution = list(map(int,self.solution))
        return [tuple(solution[i:i+2]) for i in range(0,self.num_genes,2)]
    
    def produce_sol_grid(self):
        sol = self.convert_solution()
        grid = np.copy(self.map.get_grid())
        for i in sol:
            grid[i[::-1]] = 5
        return grid

## Quick Testing Vector

In [418]:
a = Vector(1,2)
b = Vector(1,2)
assert a * b == 5
assert (a + b).x == 2
assert (a + b).y == 4
assert (a - b).x == 0
assert (a - b).y == 0
assert Vector(3,4).distance() == 5
assert Vector(4,4).angle() == 45

## Quick Testing Map

In [426]:
obstacles = [(7,0), (8, 0), (9, 0), (4, 3), (0, 4), (1, 4), \
             (3, 4), (4, 4), (0, 5), (1, 5), (3, 5), (4, 5), \
             (7, 7), (8, 7), (7, 8), (8, 8)]
shape = (10,10)
MAP = Map(start=(0,0), end=(9,9), obstacles=obstacles, shape=shape)

In [427]:
MAP.get_grid()
ga = GA_Solver(MAP, generations=50, parents_mating=20, solutions_per_pop=200, num_nodes=2, thresh=(0,10), mutation_percent=20)

In [428]:
ga.train_ga()

incorrect node selection: 0
  Obstructed at point (4, 3).
  Obstructed at point (4, 4).
  Obstructed at point (8, 7).
penalty of (0, 0) to (5, 4): 3
penalty of (5, 4) to (6, 1): 1
penalty of (6, 1) to (9, 9): 2

incorrect node selection: 1
  Obstructed at point (4, 3).
  Obstructed at point (4, 4).
  Obstructed at point (7, 7).
  Obstructed at point (8, 7).
penalty of (0, 0) to (8, 7): 4
penalty of (8, 7) to (6, 1): 0
penalty of (6, 1) to (9, 9): 2

incorrect node selection: 0
  Obstructed at point (0, 4).
  Obstructed at point (0, 5).
penalty of (0, 0) to (0, 6): 2
penalty of (0, 6) to (6, 6): 0
penalty of (6, 6) to (9, 9): 4

incorrect node selection: 0
penalty of (0, 0) to (4, 9): 0
penalty of (4, 9) to (5, 4): 0
penalty of (5, 4) to (9, 9): 3

incorrect node selection: 1
penalty of (0, 0) to (2, 1): 0
penalty of (2, 1) to (7, 7): 5
penalty of (7, 7) to (9, 9): 3

incorrect node selection: 0
  Obstructed at point (3, 4).
  Obstructed at point (4, 4).
  Obstructed at point (3, 5).
  

penalty of (7, 7) to (9, 9): 3

incorrect node selection: 1
  Obstructed at point (3, 4).
  Obstructed at point (3, 5).
  Obstructed at point (4, 5).
penalty of (0, 0) to (3, 4): 3
penalty of (3, 4) to (7, 6): 3
penalty of (7, 6) to (9, 9): 3

incorrect node selection: 0
  Obstructed at point (0, 4).
  Obstructed at point (0, 5).
penalty of (0, 0) to (0, 1): 2
penalty of (0, 1) to (1, 9): 3
penalty of (1, 9) to (9, 9): 0

incorrect node selection: 1
  Obstructed at point (1, 4).
  Obstructed at point (1, 5).
penalty of (0, 0) to (3, 9): 2
penalty of (3, 9) to (9, 9): 0
penalty of (9, 9) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 1): 0
penalty of (6, 1) to (5, 6): 0
penalty of (5, 6) to (9, 9): 3

incorrect node selection: 1
  Obstructed at point (0, 4).
  Obstructed at point (0, 5).
penalty of (0, 0) to (0, 5): 2
penalty of (0, 5) to (0, 6): 0
penalty of (0, 6) to (9, 9): 2

incorrect node selection: 0
penalty of (0, 0) to (7, 3): 0
penalty of (7, 3) to (7, 3): 


incorrect node selection: 1
penalty of (0, 0) to (9, 2): 0
penalty of (9, 2) to (8, 7): 2
penalty of (8, 7) to (9, 9): 1

incorrect node selection: 0
  Obstructed at point (7, 0).
  Obstructed at point (8, 0).
  Obstructed at point (9, 0).
penalty of (0, 0) to (4, 0): 3
penalty of (4, 0) to (8, 9): 3
penalty of (8, 9) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (1, 4).
penalty of (0, 0) to (3, 7): 1
penalty of (3, 7) to (2, 8): 0
penalty of (2, 8) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (3, 5).
penalty of (0, 0) to (4, 8): 1
penalty of (4, 8) to (3, 9): 0
penalty of (3, 9) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (4, 3).
penalty of (0, 0) to (8, 5): 1
penalty of (8, 5) to (3, 8): 0
penalty of (3, 8) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (4, 3).
penalty of (0, 0) to (7, 5): 1
penalty of (7, 5) to (3, 3): 2
penalty of (3, 3) to (9, 9): 8

incorrect node selection: 0
  Obstructed at point (4, 3).

penalty of (5, 5) to (9, 1): 1
penalty of (9, 1) to (9, 9): 0

incorrect node selection: 1
  Obstructed at point (4, 3).
  Obstructed at point (3, 4).
  Obstructed at point (4, 4).
  Obstructed at point (4, 5).
  Obstructed at point (7, 7).
  Obstructed at point (8, 7).
  Obstructed at point (7, 8).
  Obstructed at point (8, 8).
penalty of (0, 0) to (6, 6): 8
penalty of (6, 6) to (9, 9): 4
penalty of (9, 9) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (9, 2): 0
penalty of (9, 2) to (5, 9): 0
penalty of (5, 9) to (9, 9): 0

incorrect node selection: 1
  Obstructed at point (0, 4).
  Obstructed at point (0, 5).
penalty of (0, 0) to (0, 2): 2
penalty of (0, 2) to (1, 5): 3
penalty of (1, 5) to (9, 9): 2

incorrect node selection: 1
  Obstructed at point (3, 4).
  Obstructed at point (4, 4).
  Obstructed at point (4, 5).
  Obstructed at point (7, 8).
penalty of (0, 0) to (4, 5): 4
penalty of (4, 5) to (8, 9): 3
penalty of (8, 9) to (9, 9): 0

incorrect node selection: 0
p

penalty of (0, 0) to (6, 2): 0
penalty of (6, 2) to (9, 1): 0
penalty of (9, 1) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (4, 3).
penalty of (0, 0) to (5, 3): 1
penalty of (5, 3) to (0, 9): 3
penalty of (0, 9) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 1): 0
penalty of (9, 1) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (4, 3).
penalty of (0, 0) to (3, 2): 1
penalty of (3, 2) to (5, 9): 3
penalty of (5, 9) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (9, 2): 0
penalty of (9, 2) to (5, 9): 0
penalty of (5, 9) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (9, 2): 0
penalty of (9, 2) to (3, 8): 0
penalty of (3, 8) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (7, 1): 0
penalty of (7, 1) to (3, 8): 1
penalty of (3, 8) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (0, 4).
  Obstructed at point (0, 5).
penalty of (0, 0) to (0, 2

...

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 6): 0
penalty of (9, 6) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (4, 3).
penalty of (0, 0) to (6, 4): 1
penalty of (6, 4) to (9, 7): 1
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 5): 0
penalty of (9, 5) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (6, 7): 0
penalty of (6, 7) to (9, 9): 3

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (3, 7): 1
penalty of (3, 7) to (9, 9): 2

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 7): 0
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (7, 3): 0
penalty of (7, 3) to (9, 7): 0
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (4, 3).
  Obstructed at point (3, 4).
  Obstructed at point (4, 4)

penalty of (1, 3) to (9, 7): 4
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 1): 0
penalty of (9, 1) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (5, 7): 0
penalty of (5, 7) to (9, 9): 2

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (0, 7): 3
penalty of (0, 7) to (9, 9): 1

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 7): 0
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (3, 4).
  Obstructed at point (3, 5).
  Obstructed at point (4, 5).
penalty of (0, 0) to (6, 8): 3
penalty of (6, 8) to (9, 7): 4
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 1
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (7, 7): 2
penalty of (7, 7) to (9, 9): 3

incorrect node selection: 1
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (7, 7): 2
penalty of (7, 7) to (9, 

penalty of (2, 3) to (9, 7): 3
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (8, 3): 0
penalty of (8, 3) to (9, 7): 0
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 7): 0
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (2, 7): 3
penalty of (2, 7) to (9, 9): 1

incorrect node selection: 0
  Obstructed at point (3, 4).
  Obstructed at point (4, 4).
  Obstructed at point (4, 5).
  Obstructed at point (7, 8).
penalty of (0, 0) to (6, 7): 4
penalty of (6, 7) to (9, 7): 2
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 4): 0
penalty of (9, 4) to (9, 9): 0

incorrect node selection: 1
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 0): 2
penalty of (9, 0) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (9, 3): 0
penalty of (9, 3) to (9, 7

  Obstructed at point (3, 4).
  Obstructed at point (4, 4).
  Obstructed at point (4, 5).
  Obstructed at point (7, 7).
  Obstructed at point (8, 7).
  Obstructed at point (7, 8).
  Obstructed at point (8, 8).
penalty of (0, 0) to (6, 6): 8
penalty of (6, 6) to (9, 7): 2
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (8, 3): 0
penalty of (8, 3) to (9, 7): 0
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (3, 4).
  Obstructed at point (3, 5).
  Obstructed at point (4, 5).
penalty of (0, 0) to (2, 3): 3
penalty of (2, 3) to (9, 7): 3
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 8): 1
penalty of (9, 8) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (6, 7): 0
penalty of (6, 7) to (9, 9): 3

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 2): 0
penalty of (9, 2) to (9, 9):

penalty of (6, 6) to (9, 7): 2
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (3, 7): 1
penalty of (3, 7) to (9, 9): 2

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 7): 0
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 3): 0
penalty of (6, 3) to (9, 7): 0
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
  Obstructed at point (3, 4).
  Obstructed at point (3, 5).
  Obstructed at point (4, 5).
penalty of (0, 0) to (6, 9): 3
penalty of (6, 9) to (9, 7): 3
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 1
  Obstructed at point (4, 3).
penalty of (0, 0) to (4, 3): 1
penalty of (4, 3) to (9, 7): 1
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (6, 2): 0
penalty of (6, 2) to (9, 7): 0
penalty of (9, 7) to (9, 9): 0

incorrect node selection: 0
penalty of (0, 0) to (8, 3): 0
penalty of (8, 3) to (9, 7

In [429]:
ga.convert_solution()

[(6, 3), (9, 7)]

In [423]:
ga.map.get_grid()

array([[2, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 4]], dtype=uint8)

In [424]:
ga.produce_sol_grid()

array([[2, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 5, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 5],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 4]], dtype=uint8)

In [425]:
ga.fitness_func( [4,2,5,2], 0)

incorrect node selection: 0
penalty of (0, 0) to (4, 2): 0
penalty of (4, 2) to (5, 2): 0
penalty of (5, 2) to (9, 9): 2



-199999997.93119767

In [375]:
ga.solution_fitness

-99999998.9307835